In [1]:
import os
import io
import time
import pickle
import pandas as pd
import numpy as np
import cython
from numba import njit
from joblib import Parallel, delayed
import shutil

import cv2 as cv
from skimage import transform
import matplotlib.pyplot as plt
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP


image_dir = r"C:\Users\Leo's PC\Desktop\images"
csv_dir = r"C:\Users\Leo's PC\Desktop\AVA.txt"

device = 'cuda:0'

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
def edgeDetection(data):
    out = cv.Canny(data, threshold1=100, threshold2=150)
    out = np.reshape(out, (data.shape[0], data.shape[1]))
    return out

def convert2HSL(data):
    try:
        return cv.cvtColor(data, cv.COLOR_BGR2HLS)
    except:
        return cv.cvtColor(cv.cvtColor(data, cv.COLOR_GRAY2BGR), cv.COLOR_BGR2HLS)

In [4]:
@njit
def hslForLoop(data, out_array):
    for idx, value in np.ndenumerate(data): 
        if idx[2] == 0: # Only enter Hue mapping when on the first value of each pixel
            # deviation = ((value % 21.25) / 21.25 * 2) - 1 
            # deviation is between -1 and 1. Representing the distance from the CENTER of that section
            if value != 0: #when it's not a b&w image w/out hue value
                out_array[int(np.floor(value / 14.92))][idx[0]][idx[1]] = 255
            # 180/12 = 14.91. np.floor(pixel[0] / 21.251) is the corresponding map index
            # 14.92 is used instead of 14.91 to avoid getting idx[12] (the luminance map) when value==179
            # [idx[0]][idx[1] points to the correct pixel location

        elif idx[2] == 1: # Only enter Luminance mapping when on the second value of each pixel
            out_array[12][idx[0]][idx[1]] = value
            # sat value to from 0 to 255 and put it at the corresponding location

        elif idx[2] == 2: # Only enter Saturation mapping when on the third value of each pixel
            out_array[13][idx[0]][idx[1]] = value
            # sat value to from 0 to 255 and put it at the corresponding location
    return out_array

In [5]:
def hslMap(data):
    out_array = np.zeros(data.shape[0] * data.shape[1] * 14, dtype='uint8').reshape(14, data.shape[0], data.shape[1])
    # create an array with (img_width * img_height * (12+1+1)) zeros and shape it into 14 maps w/ 0s.
    
    out_array = hslForLoop(data=data, out_array=out_array)
    
    for i in range(12):
        # cv.fastNlMeansDenoising(out_array[i], out_array[i], 30.0, 7, 21)
        out_array[i] = cv.blur(out_array[i],(3,3))
    out_array[12] = cv.blur(out_array[12],(15,15))
    out_array[13] = cv.blur(out_array[13],(5,5))
    
    return out_array


In [6]:
def cvFeatureMaps(data):
    # some datatype processing here
    out = np.append(hslMap(convert2HSL(data)), [edgeDetection(data)], axis=0)
    # some datatype processing here
    return out

## Test

In [9]:
image = cv.imread('Sample Images/example7.jpg')
image = cvFeatureMaps(image)

In [8]:
for i in range(len(image)):
    cv.namedWindow( "Display window", cv.WINDOW_AUTOSIZE)
    cv.imshow("Display window", image[i])
    cv.waitKey(0)

## Edge detection tool

In [15]:
image = cv.imread(r"C:\Users\Leo's PC\Desktop\RISD Summer HW\WeChat Image_20200712120050.jpg")
edge = edgeDetection(image)

cv.imwrite(r"C:\Users\Leo's PC\Desktop\RISD Summer HW\WeChat Image_20200712120050_edge.jpg", edge)

'''
cv.namedWindow( "Display window", cv.WINDOW_AUTOSIZE)
cv.imshow("Display window", edge)
cv.waitKey(0)
'''

'\ncv.namedWindow( "Display window", cv.WINDOW_AUTOSIZE)\ncv.imshow("Display window", edge)\ncv.waitKey(0)\n'

## Code to process the entire AVA dataset

In [10]:
def SalGanCV(csv_file, save_dir):
    
    csv = pd.read_csv(csv_file, sep=' ')
    img_name_array = [subfolders for subfolders in os.listdir(save_dir)]
    
    for csv_idx, row in csv.iterrows(): #traverse the enitre csv file
        img_name = row[1]
        if (str(img_name)) in img_name_array:
            img = Image.open(os.path.join(save_dir, str(img_name), '0.jpg'))
            size = np.flip(np.array(img.size))
            img = np.array(img)
            
            try:
                maps = cvFeatureMaps(img)
            except TypeError:
                print("Image reading: TypeError. Skipped")
            else:
                maps = np.reshape(maps, [15, size[0], size[1]])
                maps = np.array(maps, dtype='uint8')
                
                folder_path = os.path.join(save_dir,str(img_name))
                if os.path.exists(folder_path):
                    for i in range(15):
                        cv.imwrite(os.path.join(folder_path, str(i + 2) + '.jpg'), maps[i])
                else: print('Folder not found. Skipped.')
        if (csv_idx) % 10000 == 0: print('==>>> On CSV Index: {}'.format(csv_idx))


In [16]:
SalGanCV(csv_file=csv_dir, save_dir=r"H:\AVA Featuremap Dummy")

==>>> On CSV Index: 0
==>>> On CSV Index: 10000
==>>> On CSV Index: 20000
==>>> On CSV Index: 30000
==>>> On CSV Index: 40000
==>>> On CSV Index: 50000
==>>> On CSV Index: 60000
==>>> On CSV Index: 70000
==>>> On CSV Index: 80000
==>>> On CSV Index: 90000
==>>> On CSV Index: 100000
==>>> On CSV Index: 110000
==>>> On CSV Index: 120000
==>>> On CSV Index: 130000
==>>> On CSV Index: 140000
==>>> On CSV Index: 150000
==>>> On CSV Index: 160000
==>>> On CSV Index: 170000
==>>> On CSV Index: 180000
==>>> On CSV Index: 190000
==>>> On CSV Index: 200000
==>>> On CSV Index: 210000
==>>> On CSV Index: 220000
==>>> On CSV Index: 230000
==>>> On CSV Index: 240000
==>>> On CSV Index: 250000
